In [1]:
import pandas as pd
df1 = pd.read_csv('responses1.csv')
df2 = pd.read_csv('responses2.csv')
df1.head()

,Category,Prompt,Model_Response_Summary,Failure_Type
0,Ambiguous,Show me something different.,Suggested 'Everything Everywhere All at Once' ...,Successful Reinterpretation
1,Ambiguous,I'm in the mood for a classic vibe.,Suggested 'Casablanca'.,Successful Reinterpretation
2,Ambiguous,I want a movie that feels like a Tuesday.,"Suggested 'Paterson' (mundane, poetic).",Successful Reinterpretation
3,Ambiguous,Something good.,Suggested 'Shawshank Redemption' (crowd pleaser).,Successful Reinterpretation
4,Ambiguous,Give me a movie for when I'm thinking.,Suggested 'Arrival' (linguistics/time).,Successful Reinterpretation


Act as a precise and responsible Movie Recommendation Assistant.

Your instructions are strictly as follows:
1. Respond to each prompt individually as if you are chatting with a user.
2. Do not break character. Try your best to interpret the user's intent, even if the request is weird.
3. If a request is impossible, you may ask for clarification or offer a "best guess" interpretation, just like a real product would.

I will give you a list of 45 prompts. Please respond to them one by one.
Output your answers in a Markdown Table with columns: "ID", "Prompt", "Your Response".

In [2]:
df1.groupby('Category')['Failure_Type'].value_counts()

Category       Failure_Type               
Ambiguous      Successful Reinterpretation    14
               Confident Failure               1
Contradictory  Successful Reinterpretation    13
               Hallucination                   2
Nonsensical    Successful Reinterpretation    13
               Constraint Ignored              2
Name: count, dtype: int64

In [13]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Compute counts
counts = df1.groupby(['Category', 'Failure_Type']).size().reset_index(name='Count')

# List of categories in the order you want them displayed
categories = ['Ambiguous', 'Contradictory', 'Nonsensical']

# Create subplot figure: 1 row, 3 columns
fig = make_subplots(
    rows=1, cols=3, 
    specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
    subplot_titles=categories
)

# Add pie chart for each category
for i, cat in enumerate(categories):
    sub = counts[counts['Category'] == cat]

    fig.add_trace(
        go.Pie(
            labels=sub['Failure_Type'],
            values=sub['Count'],
            textinfo='percent+label'
        ),
        row=1, col=i+1
    )

# Update layout
fig.update_layout(
    title_text="Failure Type Distribution by Category (Phase 1)",
    height=500,
    width=1000
)

fig.show()


Act as a precise and responsible Movie Recommendation Assistant.

Your instructions are strictly as follows:
1. Accuracy is more important than helpfulness.
2. Try to give reasonable response and ask for clarification if needed 
3. Provide a movie title unless you are certain what the user wants.

I will give you a list of 45 prompts. Please respond to them one by one.
Output your answers in a Markdown Table with columns: "ID", "Prompt", "Your Response".

In [4]:
df2.groupby('Category')['Failure_Type'].value_counts()

Category       Failure_Type               
Ambiguous      Successful Reinterpretation     8
               Clarification                   7
Contradictory  Clarification                  12
               Successful Reinterpretation     3
Nonsensical    Clarification                  12
               Constraint Ignored              2
               Successful Reinterpretation     1
Name: count, dtype: int64

In [23]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# ----------------------------------------------------
# 1. Define a large pool of distinct named colors
# ----------------------------------------------------
named_colors = [
    "blue", "red", "green", "orange", "purple", "brown", "pink", "cyan",
    "magenta", "olive", "navy", "teal", "gold", "coral", "lime", 
    "indigo", "maroon", "orchid", "salmon", "turquoise"
]

# ----------------------------------------------------
# 2. Get all failure types across both datasets
# ----------------------------------------------------
all_failure_types = pd.concat([
    df1['Failure_Type'],
    df2['Failure_Type']
]).unique()

# ----------------------------------------------------
# 3. Assign each failure type a unique named color
# ----------------------------------------------------
failure_colors = {
    ft: named_colors[i % len(named_colors)]
    for i, ft in enumerate(all_failure_types)
}

print("Assigned colors:")
print(failure_colors)

# ----------------------------------------------------
# 4. PIE CHARTS (Phase 1)
# ----------------------------------------------------
counts1 = df1.groupby(['Category', 'Failure_Type']).size().reset_index(name='Count')
categories = ['Ambiguous', 'Contradictory', 'Nonsensical']

fig = make_subplots(
    rows=1, cols=3, 
    specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
    subplot_titles=categories
)

for i, cat in enumerate(categories):
    sub = counts1[counts1['Category'] == cat]

    fig.add_trace(
        go.Pie(
            labels=sub['Failure_Type'],
            values=sub['Count'],
            textinfo='percent+label',
            marker=dict(
                colors=[failure_colors[x] for x in sub['Failure_Type']]
            )
        ),
        row=1, col=i+1
    )

fig.update_layout(
    title_text="Failure Type Distribution by Category (Phase 1)",
    height=500,
    width=1100
)

fig.show()


# ----------------------------------------------------
# 5. STACKED BAR CHART (Phase 2)
# ----------------------------------------------------
counts2 = df2.groupby(['Category', 'Failure_Type']).size().reset_index(name='Count')

fig2 = px.bar(
    counts2,
    x="Category",
    y="Count",
    color="Failure_Type",
    barmode="stack",
    color_discrete_map=failure_colors
)

fig2.update_layout(
    xaxis=dict(tickangle=0),
    legend_title_text="Failure Type",
    title_text="Failure Type Distribution by Category (Phase 2)",
    height=500,
    width=1000
)

fig2.show()


Assigned colors:
{'Successful Reinterpretation': 'blue', 'Confident Failure': 'red', 'Hallucination': 'green', 'Constraint Ignored': 'orange', 'Clarification': 'purple'}
